# Research Actions

This document is research of semantic similarity between including actions sentences from different documents. 

In [1]:
# import used modules
import os
import numpy as np
import time
import pickle
from scipy.linalg import block_diag

In [2]:
# import a model and a module for semantic analys
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('../model.bin', binary=True) 
model.init_sims(replace=True)

/home/ased/anaconda3/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
# import our module for semantic analys
import sys
sys.path.append('../')
import sem_analysis as sa


Loading the model...
Processing input...


## 1. Extract Sentences From Documents And Their Preprocessing

In [4]:
path = "SectionsWithAction/"
files = list()
for i in os.listdir(path):
    if 'Sectionstext' in i:
        files.append(i)

In [5]:
sections = dict()
for j in files:
    f = open(path + j, 'r')
    new = list()
    for line in f:
        s = ''
        for i in line:
            if i != '\n':
                s = s + i
        if len(s) > 0:
            new.append(s)
    sections[j] = new
    f.close()

In [6]:
# We leave sentences only from documents of type '2_' (see types description in folder ../../Texts)
cur_sections = {i:sections[i] for i in sections.keys() if '2_' in i}

In current step we initialize a distance matrix for our sentences: an matrix element with indexes `i` and `j` is distance between `i`-th and `j`-th sentences. If sentences `i` and `j` from one document we set this element equal to $\infty$.

In [9]:
list_ = list()
keys = list(cur_sections.keys()).copy()
keys.sort()
for j in keys:
    if len(cur_sections[j]) > 0:
        list_.append(np.inf * np.ones((len(cur_sections[j]), len(cur_sections[j]))))
    else:
        cur_sections.pop(j)
        keys.pop(j)
D = block_diag(*np.array(list_))

In [10]:
phrases = list()
for j in keys:
    phrases = phrases + cur_sections[j]
print(len(phrases))

528


Now we transform our phrases to format for [RusVectores](https://rusvectores.org/ru/). See [tutorial for RusVectores](https://github.com/akutuzov/webvectors/blob/master/preprocessing/rusvectores_tutorial.ipynb).

In [11]:
if not 'sections_data.pickle' in os.listdir():
    t = time.time()
    phrases_transform = list()
    for i in phrases:
        phrases_transform.append(sa.tag_ud(i))
        if len(phrases_transform) % 100 == 0:
            print(len(phrases_transform), time.time() - t)
    with open('sections_data.pickle', 'wb') as f:
        pickle.dump(phrases_transform, f)
        f.close()
else:
    with open('sections_data.pickle', 'rb') as f: 
        phrases_transform = pickle.load(f)
        f.close()

100 479.19846844673157
200 949.2676832675934
300 1420.5151534080505
400 1887.7130465507507
500 2364.801566362381


## 2. Similarity

### 2.1. Similarity withoud filtration

In [12]:
# Distance Matrix
import time
t = time.time()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i, j] != np.inf:
            D[i,j] = model.wmdistance(phrases_transform[i], phrases_transform[j])
            D[j, i] = D[i, j]
            if i == 0 and j == 100:
                print((time.time() - t)/100*D.shape[0]*D.shape[0] * 0.9 * 0.5)

367.6293245544434


#### Output

In [13]:
sim = dict()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i,j] != np.inf and not np.isnan(D[i,j]):
            if not sim.__contains__(D[i,j]):
                sim[D[i,j]] = []
            sim[D[i,j]].append((i, j, phrases[i], phrases[j]))

In [14]:
keys = list(sim.keys())
keys.sort()

In [15]:
def print_eps(eps, keys, mydict):
    i = 0
    #while keys[i] > keys[0] - eps and i < len(keys):
    while keys[i] < eps and i < len(keys):
        if  keys[i] > 0:
            for item in mydict[keys[i]]:
                print(keys[i], item[0],':',item[2],'\t',item[1],':', item[3])
        i += 1

In [16]:
print_eps(0.6, keys, sim)

0.15954787559479477 38 : ListSection  Переспросите год выпуска и пробег авто 	 196 : ListSection   Одометр Соответствие года выпуска и пробега автомобиля.
0.29519697349665824 196 : ListSection   Одометр Соответствие года выпуска и пробега автомобиля. 	 230 : Section Автомобили 2007-2012 года выпуска
0.29519697349665824 196 : ListSection   Одометр Соответствие года выпуска и пробега автомобиля. 	 231 : Section  Автомобили 2007-2012 года выпуска 
0.29800054613915083 77 : Section Как правильно выбрать автомобиль 	 275 : Section Как выбрать хороший автомобиль
0.29800054613915083 188 : Section Как правильно выбрать автомобиль 	 275 : Section Как выбрать хороший автомобиль
0.3431498737350106 196 : ListSection   Одометр Соответствие года выпуска и пробега автомобиля. 	 363 : ListSection  несоответствием года выпуска и пробега.
0.34496041212683914 77 : Section Как правильно выбрать автомобиль 	 195 : Section ﻿Как выбрать автомобиль с пробегом
0.34496041212683914 188 : Section Как правильно выб

### 2.2. SImilarity With Filtration

In [17]:
import pickle
def transform(phrase):
    def cond(word):
        _ = ['_NOUN', '_VERB','_ADV', '_ADJ']
        for i in _:
            if i in word:
                return True
        return False
    phrase = phrase.copy()
    for i in phrase:
        if not cond(i):
            phrase.remove(i)
    return phrase

with open('action_data.pickle', 'rb') as f: 
    phrases_transform = pickle.load(f)
new = list()
for i in phrases_transform:
    new.append(transform(i))

In [18]:
import time
t = time.time()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i, j] != np.inf:
            D[i,j] = model.wmdistance(new[i], new[j])
            D[j, i] = D[i, j]
            if i == 0 and j == 100:
                print((time.time() - t)/100*D.shape[0]*D.shape[0] * 0.9 * 0.5)

236.5562037963867


#### Output

In [19]:
sim_filtr = dict()
for i in range(D.shape[0]):
    for j in range(i+1, D.shape[1]):
        if D[i,j] != np.inf and not np.isnan(D[i,j]):
            if not sim_filtr.__contains__(D[i,j]):
                sim_filtr[D[i,j]] = []
            sim_filtr[D[i,j]].append((i, j, phrases[i], phrases[j]))

In [20]:
keys = list(sim_filtr.keys())
keys.sort()

In [21]:
def print_eps(eps, keys, mydict):
    i = 0
    while keys[i] < eps and i < len(keys):
        if  keys[i] > 0:
            for item in mydict[keys[i]]:
                print(keys[i], item[0],':',item[2],'\t',item[1],':', item[3])
        i += 1

In [22]:
print_eps(0.6, keys, sim_filtr)

0.4224470385429383 182 : ListSection  На машине стоят либо разные стекла, либо стекла года выпуска, отличные от года выпуска самого автомобиля 	 193 : ListSection  Кондиционер и печку нужно проверить отдельно.
0.4326904104550839 78 : ListSection  Вид автомобиля должен соответствовать его возрасту. 	 92 : ListSection  Как она будет оформляться при продаже?
0.47209424908647535 266 : ListSection  Рассчитайте самостоятельно, какой приблизительно должен быть пробег у выбранного легкового автомобиля. 	 481 : ListSection  Принюхайтесь.
0.4778520528302073 157 : ListSection  Откройте крышку горловины для залива масла и осмотрите ее, поцарапайте внутреннюю поверхность крышки отверткой. 	 191 : ListSection  Задние и передние колёса с одной стороны автомобиля должны находиться на одной линии, – это легко увидеть с расстояния 3-4 метров.
0.5372836488081919 405 : Section Двигатель 	 412 : Section Первым делом
0.5641961295699341 405 : Section Двигатель 	 523 : ListSection  Не ставьте свою подпись в П